In [52]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 100)
sns.set(style='whitegrid')

# Define paths
DATA_DIR = "../data/raw"
PROCESSED_DIR = "../data/processed"

# Logging
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logger.info("Notebook started.")

INFO:__main__:Notebook started.


In [53]:
file_path_desc = os.path.join(DATA_DIR, "call-report-data-2024-03/AcctDesc.txt") 

In [54]:
try:
    acct_description = pd.read_csv(file_path_desc)
    logger.info(f"Loaded dataset with shape {acct_description.shape}")
except Exception as e:
    logger.error(f"Failed to load file: {e}")

INFO:__main__:Loaded dataset with shape (3377, 12)


In [55]:
acct_description.head()
acct_description.columns

,Account,AcctName,AcctDesc,TableName,Annualize,CBSAccount,FPRAccount,STATAcctMain,STATAcctState,STATAcctGraph,STATAcctTable8,STATUS
0,Acct_001,Amount - Total Other Loans (Purchased or to no...,Include the dollar amount of outstanding loans...,FS220A,False,True,True,True,True,False,False,Inactive
1,Acct_002,Amount of Leases Receivable,"Leases Receivable. Include the rate, number, a...",FS220A,False,False,False,False,False,False,False,Active
2,Acct_003,Loans Held for Sale,Loans Held for Sale. Report the amount of outs...,FS220C,False,False,False,False,False,False,False,Active
3,Acct_007,Land and Building,"Land and building, less depreciation on buildi...",FS220,False,True,True,True,True,False,True,Active
4,Acct_008,Other Fixed Assets,All other fixed assets such as furniture and f...,FS220,False,True,True,True,True,False,True,Active


Index(['Account', 'AcctName', 'AcctDesc', 'TableName', 'Annualize',
       'CBSAccount', 'FPRAccount', 'STATAcctMain', 'STATAcctState',
       'STATAcctGraph', 'STATAcctTable8', 'STATUS'],
      dtype='object')

### Column Name	Meaning

**CBSAccount**	Indicates whether the account is used in the Call Report system (CBS) — the core regulatory report that credit unions submit to the NCUA. Usually True/False.

**FPRAccount**	Flags whether this account is included in the Financial Performance Report (FPR), which summarizes key performance indicators.

**STATAcctMain**	True if the account is used in the main statistical report by the NCUA or analysts. Typically part of core metrics used for national trends.

**STATAcctState**	True if the account is used for state-level summaries or reports.

**STATAcctGraph**	True if the account is used in visualizations/graphs in published reports (like NCUA's quarterly charts).

**STATAcctTable8**	True if the account appears in Table 8 of NCUA’s standard reporting tables (one of the summary tables presented each quarter).

In [56]:
acct_map = acct_description.set_index('Account')['AcctName'].dropna().to_dict()

In [57]:
#acct_map

In [58]:

def load_fs220_files(data_dir: str, subfolder: str, base_name: str = "FS220", suffixes: list = None) -> dict:
    """
    Loads a set of FS220-related.txt files into a dictionary of DataFrames with logging and encoding fallback.

    Parameters:
    - data_dir (str): Base directory path containing the data folder.
    - subfolder (str): Subdirectory name where the text files are located.
    - base_name (str): Common prefix of the files, default is 'FS220'.
    - suffixes (list): List of suffixes to load (e.g., ["", "A", "B", ...])

    Returns:
    - dict: Dictionary with keys as variable names and values as pandas DataFrames
    """

    # Setup log file
    os.makedirs("logs", exist_ok=True)
    logging.basicConfig(
        filename="logs/data_load.log",
        filemode="a",  # Append mode for multiple call multiple 
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s"
    )

    file_path = os.path.join(data_dir, subfolder)
    suffixes = suffixes if suffixes is not None else [""]

    dataframes = {}

    for suffix in suffixes:
        filename = f"{base_name}{suffix}.txt" if suffix else f"{base_name}.txt"
        full_path = os.path.join(file_path, filename)
        df_name = f"{base_name.lower()}_{suffix.lower()}" if suffix else base_name.lower()

        try:
            df = pd.read_csv(full_path, encoding="utf-8")
            logging.info(f"Loaded: {filename} with UTF-8 → variable '{df_name}' with shape {df.shape}")
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(full_path, encoding="latin1")
                logging.info(f"Loaded: {filename} with Latin-1 → variable '{df_name}' with shape {df.shape}")
            except Exception as e:
                logging.error(f"Encoding error loading {filename}: {e}")
                continue
        except FileNotFoundError:
            logging.warning(f"File not found: {filename}")
            continue
        except Exception as e:
            logging.error(f"General error loading {filename}: {e}")
            continue

        dataframes[df_name] = df
                                        
    return dataframes                                        

In [59]:
SUBFOLDER = "call-report-data-2024-03"
SUFFIXES = ["", "A", "B", "C", "D", "G", "H", "I", "J", "K", "L", "M", "N", "P", "Q", "R", "S"]

fs220_dataframes = load_fs220_files(DATA_DIR, SUBFOLDER, base_name="FS220", suffixes=SUFFIXES)

INFO:root:Loaded: FS220.txt with UTF-8 → variable 'fs220' with shape (4670, 246)
INFO:root:Loaded: FS220A.txt with UTF-8 → variable 'fs220_a' with shape (4670, 249)
INFO:root:Loaded: FS220B.txt with UTF-8 → variable 'fs220_b' with shape (4670, 250)
INFO:root:Loaded: FS220C.txt with UTF-8 → variable 'fs220_c' with shape (4670, 250)
INFO:root:Loaded: FS220D.txt with Latin-1 → variable 'fs220_d' with shape (4670, 230)
INFO:root:Loaded: FS220G.txt with UTF-8 → variable 'fs220_g' with shape (4670, 252)
INFO:root:Loaded: FS220H.txt with UTF-8 → variable 'fs220_h' with shape (4670, 249)
INFO:root:Loaded: FS220I.txt with UTF-8 → variable 'fs220_i' with shape (4670, 251)
INFO:root:Loaded: FS220J.txt with UTF-8 → variable 'fs220_j' with shape (4670, 255)
INFO:root:Loaded: FS220K.txt with UTF-8 → variable 'fs220_k' with shape (4670, 148)
INFO:root:Loaded: FS220L.txt with UTF-8 → variable 'fs220_l' with shape (4670, 243)
INFO:root:Loaded: FS220M.txt with UTF-8 → variable 'fs220_m' with shape (4670

In [62]:
fs220= fs220_dataframes['fs220']